In [1]:
import torch
import torch.nn as nn

from seisLM.data_pipeline import foreshock_aftershock_dataloaders as loaders
from seisLM.model.task_specific import foreshock_aftershock_models as models

dataloaders = loaders.prepare_foreshock_aftershock_dataloaders(
  num_classes=2,
  batch_size=32,
  event_split_method='temporal'
)

Seed set to 42
Seed set to 42


In [2]:
# Example usage
# Input shape: (batch_size, 3, 2500) where 3 is the number of channels
# and 2500 is the time length
# model = models.Conv1DShockClassifier(
#   in_channels=3,
#   num_classes=2,
#   num_layers=7,
#   initial_filters=32,
#   kernel_size=3,
# )


import torch
import torch.nn as nn
import torch.nn.functional as F

class BasicBlock1D(nn.Module):
    def __init__(self, in_channels, out_channels, stride=1, downsample=None):
        super(BasicBlock1D, self).__init__()
        self.conv1 = nn.Conv1d(in_channels, out_channels, kernel_size=3, stride=stride, padding=1, bias=False)
        self.bn1 = nn.BatchNorm1d(out_channels)
        self.relu = nn.ReLU(inplace=True)
        self.conv2 = nn.Conv1d(out_channels, out_channels, kernel_size=3, stride=1, padding=1, bias=False)
        self.bn2 = nn.BatchNorm1d(out_channels)
        self.downsample = downsample

    def forward(self, x):
        identity = x
        if self.downsample is not None:
            identity = self.downsample(x)

        out = self.conv1(x)
        out = self.bn1(out)
        out = self.relu(out)
        out = self.conv2(out)
        out = self.bn2(out)

        out += identity
        out = self.relu(out)

        return out

class ResNet1D(nn.Module):
    def __init__(self, block, layers, num_classes=1000):
        super(ResNet1D, self).__init__()
        self.in_channels = 64
        self.conv1 = nn.Conv1d(3, 64, kernel_size=7, stride=2, padding=3, bias=False)
        self.bn1 = nn.BatchNorm1d(64)
        self.relu = nn.ReLU(inplace=True)
        self.maxpool = nn.MaxPool1d(kernel_size=3, stride=2, padding=1)
        self.layer1 = self._make_layer(block, 64, layers[0])
        self.layer2 = self._make_layer(block, 128, layers[1], stride=2)
        self.layer3 = self._make_layer(block, 256, layers[2], stride=2)
        self.layer4 = self._make_layer(block, 512, layers[3], stride=2)
        self.avgpool = nn.AdaptiveAvgPool1d(1)
        self.fc = nn.Linear(512, num_classes)

    def _make_layer(self, block, out_channels, blocks, stride=1):
        downsample = None
        if stride != 1 or self.in_channels != out_channels:
            downsample = nn.Sequential(
                nn.Conv1d(self.in_channels, out_channels, kernel_size=1, stride=stride, bias=False),
                nn.BatchNorm1d(out_channels),
            )

        layers = []
        layers.append(block(self.in_channels, out_channels, stride, downsample))
        self.in_channels = out_channels
        for _ in range(1, blocks):
            layers.append(block(self.in_channels, out_channels))

        return nn.Sequential(*layers)

    def forward(self, x):
        x = self.conv1(x)
        x = self.bn1(x)
        x = self.relu(x)
        x = self.maxpool(x)

        x = self.layer1(x)
        x = self.layer2(x)
        x = self.layer3(x)
        x = self.layer4(x)

        x = self.avgpool(x)
        x = torch.flatten(x, 1)
        x = self.fc(x)

        return x

def resnet18_1d(num_classes):
    return ResNet1D(BasicBlock1D, [2, 2, 2, 2], num_classes)

# Example usage
model = resnet18_1d(num_classes=10)  # For 10 classes
print(model)



In [3]:
from torchinfo import summary

summary(model, input_size=(32, 3, 2500))

Layer (type:depth-idx)                   Output Shape              Param #
Conv1DShockClassifier                    [32, 2]                   --
├─Sequential: 1-1                        [32, 2048, 16]            --
│    └─DoubleConvBlock: 2-1              [32, 32, 1248]            --
│    │    └─Sequential: 3-1              [32, 32, 1248]            3,488
│    └─DoubleConvBlock: 2-2              [32, 64, 622]             --
│    │    └─Sequential: 3-2              [32, 64, 622]             18,688
│    └─DoubleConvBlock: 2-3              [32, 128, 309]            --
│    │    └─Sequential: 3-3              [32, 128, 309]            74,240
│    └─DoubleConvBlock: 2-4              [32, 256, 153]            --
│    │    └─Sequential: 3-4              [32, 256, 153]            295,936
│    └─DoubleConvBlock: 2-5              [32, 512, 75]             --
│    │    └─Sequential: 3-5              [32, 512, 75]             1,181,696
│    └─DoubleConvBlock: 2-6              [32, 1024, 36]       

In [4]:
train_loader = dataloaders['train']

batch = next(iter(train_loader))

In [5]:
out = model(batch[0].cuda())